# Stream Ensemble Classification
---

## `NEWeather` dataset

**Description:** The National Oceanic and Atmospheric Administration (NOAA),
has compiled a database of weather measurements from over 7,000 weather 
stations worldwide. Records date back to the mid-1900s. Daily measurements
include a variety of features (temperature, pressure, wind speed, etc.) as
well as a series of indicators for precipitation and other weather-related
events. The `NEweather` dataset contains data from this database, specifically
from the Offutt Air Force Base in Bellevue, Nebraska ranging for over 50 years
(1949-1999).

**Features:** 8 Daily weather measurements
 
|       Attribute      | Description |
|:--------------------:|:-----------------------------|
| `temp`                   | Temperature
| `dew_pnt`                | Dew Point
| `sea_lvl_press`          | Sea Level Pressure
| `visibility`             | Visibility
| `avg_wind_spd`           | Average Wind Speed
| `max_sustained_wind_spd` | Maximum Sustained Wind Speed
| `max_temp`               | Maximum Temperature
| `min_temp`               | Minimum Temperature


**Class:** `rain` | 0: no rain, 1: rain
 
**Samples:** 18,159


In [1]:
import pandas as pd
from river.stream import iter_pandas
from river.metrics import Accuracy,BalancedAccuracy,CohenKappa,GeometricMean
from river.metrics.base import Metrics
from river.utils import Rolling
from river.evaluate import progressive_val_score

In [2]:
data = pd.read_csv("../datasets/NEweather.csv")
features = data.columns[:-1]

## Online Bagging

---
[Online Bagging](https://riverml.xyz/latest/api/ensemble/BaggingClassifier/) is an Online bootstrap aggregation for classification.

In [3]:
from river.ensemble import BaggingClassifier
from river.tree import HoeffdingTreeClassifier

model = BaggingClassifier(model=HoeffdingTreeClassifier(),
                          n_models=10,
                          seed=42)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 70.77%, BalancedAccuracy: 67.47%, GeometricMean: 66.80%, CohenKappa: 34.45%
[2,000] Accuracy: 70.29%, BalancedAccuracy: 66.94%, GeometricMean: 66.42%, CohenKappa: 32.37%
[3,000] Accuracy: 71.29%, BalancedAccuracy: 62.56%, GeometricMean: 59.04%, CohenKappa: 26.45%
[4,000] Accuracy: 71.82%, BalancedAccuracy: 61.73%, GeometricMean: 56.61%, CohenKappa: 25.72%
[5,000] Accuracy: 72.67%, BalancedAccuracy: 62.70%, GeometricMean: 57.67%, CohenKappa: 27.97%
[6,000] Accuracy: 73.18%, BalancedAccuracy: 63.95%, GeometricMean: 59.71%, CohenKappa: 30.34%
[7,000] Accuracy: 74.10%, BalancedAccuracy: 65.01%, GeometricMean: 61.06%, CohenKappa: 32.60%
[8,000] Accuracy: 73.83%, BalancedAccuracy: 64.93%, GeometricMean: 61.02%, CohenKappa: 32.39%
[9,000] Accuracy: 73.95%, BalancedAccuracy: 64.68%, GeometricMean: 60.30%, CohenKappa: 32.20%
[10,000] Accuracy: 74.11%, BalancedAccuracy: 64.59%, GeometricMean: 60.03%, CohenKappa: 32.14%
[11,000] Accuracy: 74.56%, BalancedAccuracy: 64.71%, Geomet

Accuracy: 74.68%, BalancedAccuracy: 66.38%, GeometricMean: 62.53%, CohenKappa: 35.78%

## Leveraging Bagging
---

[Leveraging Bagging](https://riverml.xyz/latest/api/ensemble/LeveragingBaggingClassifier/) is an improvement over the Oza Bagging algorithm. The bagging performance is leveraged by increasing the re-sampling. It uses a poisson distribution to simulate the re-sampling process. To increase re-sampling it uses a higher w value of the Poisson distribution (agerage number of events), 6 by default, increasing the input space diversity, by attributing a different range of weights to the data samples.

To deal with concept drift, Leveraging Bagging uses the ADWIN algorithm to monitor the performance of each member of the enemble If concept drift is detected, the worst member of the ensemble (based on the error estimation by ADWIN) is replaced by a new (empty) classifier.

In [4]:
from river.ensemble import LeveragingBaggingClassifier
from river.tree import HoeffdingTreeClassifier

model = LeveragingBaggingClassifier(model=HoeffdingTreeClassifier(),
                          n_models=10,
                          seed=42)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 76.58%, BalancedAccuracy: 73.20%, GeometricMean: 72.56%, CohenKappa: 46.56%
[2,000] Accuracy: 78.29%, BalancedAccuracy: 71.99%, GeometricMean: 70.25%, CohenKappa: 45.95%
[3,000] Accuracy: 78.99%, BalancedAccuracy: 71.33%, GeometricMean: 68.98%, CohenKappa: 45.50%
[4,000] Accuracy: 78.69%, BalancedAccuracy: 70.72%, GeometricMean: 67.99%, CohenKappa: 44.73%
[5,000] Accuracy: 78.52%, BalancedAccuracy: 70.52%, GeometricMean: 67.71%, CohenKappa: 44.39%
[6,000] Accuracy: 78.66%, BalancedAccuracy: 71.28%, GeometricMean: 68.88%, CohenKappa: 45.51%
[7,000] Accuracy: 79.07%, BalancedAccuracy: 71.93%, GeometricMean: 69.76%, CohenKappa: 46.65%
[8,000] Accuracy: 78.57%, BalancedAccuracy: 71.57%, GeometricMean: 69.40%, CohenKappa: 45.79%
[9,000] Accuracy: 78.41%, BalancedAccuracy: 71.11%, GeometricMean: 68.69%, CohenKappa: 45.17%
[10,000] Accuracy: 78.56%, BalancedAccuracy: 71.08%, GeometricMean: 68.57%, CohenKappa: 45.24%
[11,000] Accuracy: 78.72%, BalancedAccuracy: 71.07%, Geomet

Accuracy: 78.71%, BalancedAccuracy: 72.68%, GeometricMean: 70.85%, CohenKappa: 47.86%

## AdaptiveRandomForest
---



The 3 most important aspects of [ARF](https://riverml.xyz/latest/api/ensemble/AdaptiveRandomForestClassifier/) are:
- inducing diversity through re-sampling
- inducing diversity through randomly selecting subsets of features for node splits
- drift detectors per base tree, which cause selective resets in response to drifts

It also allows training background trees, which start training if a warning is detected and replace the active tree if the warning escalates to a drift.

In [6]:
from river.forest import ARFClassifier

model = ARFClassifier(n_models=10)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 74.97%, BalancedAccuracy: 67.35%, GeometricMean: 63.67%, CohenKappa: 37.88%
[2,000] Accuracy: 76.54%, BalancedAccuracy: 67.11%, GeometricMean: 62.85%, CohenKappa: 37.97%
[3,000] Accuracy: 77.53%, BalancedAccuracy: 67.12%, GeometricMean: 62.42%, CohenKappa: 38.51%
[4,000] Accuracy: 77.44%, BalancedAccuracy: 67.20%, GeometricMean: 62.38%, CohenKappa: 38.82%
[5,000] Accuracy: 77.60%, BalancedAccuracy: 67.48%, GeometricMean: 62.70%, CohenKappa: 39.46%
[6,000] Accuracy: 77.65%, BalancedAccuracy: 68.13%, GeometricMean: 63.91%, CohenKappa: 40.45%
[7,000] Accuracy: 78.03%, BalancedAccuracy: 68.75%, GeometricMean: 64.86%, CohenKappa: 41.59%
[8,000] Accuracy: 77.76%, BalancedAccuracy: 68.74%, GeometricMean: 64.95%, CohenKappa: 41.42%
[9,000] Accuracy: 77.74%, BalancedAccuracy: 68.69%, GeometricMean: 64.78%, CohenKappa: 41.44%
[10,000] Accuracy: 77.89%, BalancedAccuracy: 68.73%, GeometricMean: 64.79%, CohenKappa: 41.58%
[11,000] Accuracy: 78.20%, BalancedAccuracy: 68.95%, Geomet

Accuracy: 77.89%, BalancedAccuracy: 70.43%, GeometricMean: 67.52%, CohenKappa: 44.32%

## StreamingRandomPatches
---
[SRP](https://riverml.xyz/latest/api/ensemble/SRPClassifier/) is an ensemble method that simulates bagging or random subspaces. The default algorithm uses both bagging and random subspaces, namely Random Patches. The default base estimator is a Hoeffding Tree, but other base estimators can be used (differently from random forest variations).

In [7]:
from river.ensemble import SRPClassifier
from river.tree import HoeffdingTreeClassifier

model = SRPClassifier(model=HoeffdingTreeClassifier(),
                      n_models=10,
                      seed=42)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream, 
                      model=model, 
                      metric=metrics, 
                      print_every=1000)

[1,000] Accuracy: 72.97%, BalancedAccuracy: 65.15%, GeometricMean: 61.14%, CohenKappa: 33.02%
[2,000] Accuracy: 75.09%, BalancedAccuracy: 65.36%, GeometricMean: 60.68%, CohenKappa: 34.10%
[3,000] Accuracy: 76.53%, BalancedAccuracy: 65.39%, GeometricMean: 59.82%, CohenKappa: 35.02%
[4,000] Accuracy: 76.84%, BalancedAccuracy: 66.08%, GeometricMean: 60.65%, CohenKappa: 36.63%
[5,000] Accuracy: 76.92%, BalancedAccuracy: 66.32%, GeometricMean: 60.95%, CohenKappa: 37.13%
[6,000] Accuracy: 77.28%, BalancedAccuracy: 67.00%, GeometricMean: 61.96%, CohenKappa: 38.52%
[7,000] Accuracy: 77.81%, BalancedAccuracy: 67.72%, GeometricMean: 63.02%, CohenKappa: 40.00%
[8,000] Accuracy: 77.60%, BalancedAccuracy: 67.90%, GeometricMean: 63.44%, CohenKappa: 40.13%
[9,000] Accuracy: 77.72%, BalancedAccuracy: 68.02%, GeometricMean: 63.48%, CohenKappa: 40.54%
[10,000] Accuracy: 77.85%, BalancedAccuracy: 68.17%, GeometricMean: 63.71%, CohenKappa: 40.80%
[11,000] Accuracy: 78.16%, BalancedAccuracy: 68.37%, Geomet

Accuracy: 78.16%, BalancedAccuracy: 70.02%, GeometricMean: 66.51%, CohenKappa: 44.12%

## Concept Drift Impact

Concept drift can negatively impact learning methods if not properly handled. Multiple real-world applications suffer **model degradation** as the models can not adapt to changes in the data.

---
## `AGRAWAL` dataset

We will load the data from a csv file. The data was generated using the `AGRAWAL` data generator with 3 **gradual drifts** at the 5k, 10k, and 15k marks. It contains 9 features, 6 numeric and 3 categorical.

There are 10 functions for generating binary class labels from the features. These functions determine whether a **loan** should be approved.

| Feature    | Description            | Values                                                                |
|------------|------------------------|-----------------------------------------------------------------------|
| `salary`     | salary                 | uniformly distributed from 20k to 150k                                |
| `commission` | commission             | if (salary <   75k) then 0 else uniformly distributed from 10k to 75k |
| `age`        | age                    | uniformly distributed from 20 to 80                                   |
| `elevel`     | education level        | uniformly chosen from 0 to 4                                          |
| `car`        | car maker              | uniformly chosen from 1 to 20                                         |
| `zipcode`    | zip code of the town   | uniformly chosen from 0 to 8                                          |
| `hvalue`     | value of the house     | uniformly distributed from 50k x zipcode to 100k x zipcode            |
| `hyears`     | years house owned      | uniformly distributed from 1 to 30                                    |
| `loan`       | total loan amount      | uniformly distributed from 0 to 500k                                  |

**Class:** `y` | 0: no loan, 1: loan
 
**Samples:** 20,000

`elevel`, `car`, and `zipcode` are categorical features.

In [8]:
data = pd.read_csv("../datasets/agr_a_20k.csv")
features = data.columns[:-1]

## ADWIN Online Bagging

---
[ADWIN Online Bagging](https://riverml.xyz/latest/api/ensemble/ADWINBaggingClassifier/) is the online bagging method with the addition of the ADWIN algorithm as a change detector. If concept drift is detected, the worst member of the ensemble (based on the error estimation by ADWIN) is replaced by a new (empty) classifier.

In [9]:
from river.ensemble import ADWINBaggingClassifier
from river.tree import HoeffdingTreeClassifier

model = ADWINBaggingClassifier(model=HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode']),
                          n_models=10,
                          seed=42)
metrics = Rolling(Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()]),window_size=500)
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=500)

[500] Accuracy: 79.16%, BalancedAccuracy: 71.24%, GeometricMean: 67.10%, CohenKappa: 47.58%
[1,000] Accuracy: 84.00%, BalancedAccuracy: 76.81%, GeometricMean: 73.77%, CohenKappa: 59.78%
[1,500] Accuracy: 88.40%, BalancedAccuracy: 86.09%, GeometricMean: 85.72%, CohenKappa: 74.05%
[2,000] Accuracy: 92.20%, BalancedAccuracy: 92.95%, GeometricMean: 92.92%, CohenKappa: 83.17%
[2,500] Accuracy: 89.00%, BalancedAccuracy: 87.97%, GeometricMean: 87.93%, CohenKappa: 75.09%
[3,000] Accuracy: 92.60%, BalancedAccuracy: 92.35%, GeometricMean: 92.35%, CohenKappa: 83.59%
[3,500] Accuracy: 92.00%, BalancedAccuracy: 92.96%, GeometricMean: 92.92%, CohenKappa: 82.07%
[4,000] Accuracy: 92.60%, BalancedAccuracy: 91.02%, GeometricMean: 90.91%, CohenKappa: 82.73%
[4,500] Accuracy: 90.20%, BalancedAccuracy: 88.54%, GeometricMean: 88.40%, CohenKappa: 77.88%
[5,000] Accuracy: 89.20%, BalancedAccuracy: 87.44%, GeometricMean: 87.26%, CohenKappa: 75.73%
[5,500] Accuracy: 52.80%, BalancedAccuracy: 55.21%, GeometricM

Accuracy: 71.20%, BalancedAccuracy: 70.50%, GeometricMean: 70.44%, CohenKappa: 40.20%

## Leveraging Bagging

In [10]:
from river.ensemble import LeveragingBaggingClassifier
from river.tree import HoeffdingTreeClassifier

model = LeveragingBaggingClassifier(model=HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode']),
                          n_models=10,
                          seed=42)
metrics = Rolling(Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()]),window_size=500)
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=500)

[500] Accuracy: 70.14%, BalancedAccuracy: 62.38%, GeometricMean: 57.80%, CohenKappa: 26.97%
[1,000] Accuracy: 76.20%, BalancedAccuracy: 68.70%, GeometricMean: 64.98%, CohenKappa: 41.08%
[1,500] Accuracy: 81.60%, BalancedAccuracy: 76.42%, GeometricMean: 74.27%, CohenKappa: 56.84%
[2,000] Accuracy: 83.60%, BalancedAccuracy: 77.74%, GeometricMean: 75.55%, CohenKappa: 60.29%
[2,500] Accuracy: 83.80%, BalancedAccuracy: 78.60%, GeometricMean: 77.22%, CohenKappa: 60.65%
[3,000] Accuracy: 84.80%, BalancedAccuracy: 79.93%, GeometricMean: 78.58%, CohenKappa: 63.66%
[3,500] Accuracy: 87.00%, BalancedAccuracy: 81.78%, GeometricMean: 80.64%, CohenKappa: 67.58%
[4,000] Accuracy: 88.60%, BalancedAccuracy: 84.52%, GeometricMean: 83.78%, CohenKappa: 72.36%
[4,500] Accuracy: 87.00%, BalancedAccuracy: 83.38%, GeometricMean: 82.62%, CohenKappa: 69.68%
[5,000] Accuracy: 89.40%, BalancedAccuracy: 87.17%, GeometricMean: 86.89%, CohenKappa: 75.94%
[5,500] Accuracy: 55.80%, BalancedAccuracy: 56.31%, GeometricM

Accuracy: 71.20%, BalancedAccuracy: 69.00%, GeometricMean: 68.37%, CohenKappa: 38.39%

## AdaptiveRandomForest

In [12]:
from river.forest import ARFClassifier

model = ARFClassifier(n_models=10,nominal_attributes=['elevel', 'car', 'zipcode'])
metrics = Rolling(Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()]),window_size=500)
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=500)

[500] Accuracy: 69.34%, BalancedAccuracy: 61.33%, GeometricMean: 56.35%, CohenKappa: 24.77%
[1,000] Accuracy: 71.00%, BalancedAccuracy: 61.17%, GeometricMean: 53.73%, CohenKappa: 25.47%
[1,500] Accuracy: 74.40%, BalancedAccuracy: 66.81%, GeometricMean: 61.39%, CohenKappa: 37.58%
[2,000] Accuracy: 75.00%, BalancedAccuracy: 65.37%, GeometricMean: 58.04%, CohenKappa: 35.57%
[2,500] Accuracy: 75.60%, BalancedAccuracy: 66.51%, GeometricMean: 61.42%, CohenKappa: 37.15%
[3,000] Accuracy: 78.80%, BalancedAccuracy: 71.55%, GeometricMean: 68.13%, CohenKappa: 47.49%
[3,500] Accuracy: 79.00%, BalancedAccuracy: 70.05%, GeometricMean: 66.06%, CohenKappa: 44.88%
[4,000] Accuracy: 80.20%, BalancedAccuracy: 71.91%, GeometricMean: 68.28%, CohenKappa: 48.89%
[4,500] Accuracy: 77.60%, BalancedAccuracy: 69.62%, GeometricMean: 65.11%, CohenKappa: 43.94%
[5,000] Accuracy: 77.80%, BalancedAccuracy: 70.07%, GeometricMean: 65.66%, CohenKappa: 44.87%
[5,500] Accuracy: 49.20%, BalancedAccuracy: 52.73%, GeometricM

Accuracy: 70.00%, BalancedAccuracy: 66.03%, GeometricMean: 63.85%, CohenKappa: 33.54%

## StreamingRandomPatches
---
We set the drift and warning detection options

In [13]:
from river.ensemble import SRPClassifier
from river.tree import HoeffdingTreeClassifier
from river.drift import ADWIN

model = SRPClassifier(model=HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode']),
                      n_models=10,
                      drift_detector=ADWIN(delta=0.001),
                      warning_detector=ADWIN(delta=0.01),
                      seed=42)

metrics = Rolling(Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()]),window_size=500)
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream, 
                      model=model, 
                      metric=metrics, 
                      print_every=500)

[500] Accuracy: 88.58%, BalancedAccuracy: 85.31%, GeometricMean: 84.74%, CohenKappa: 73.36%
[1,000] Accuracy: 94.20%, BalancedAccuracy: 93.08%, GeometricMean: 93.02%, CohenKappa: 86.82%
[1,500] Accuracy: 93.40%, BalancedAccuracy: 92.11%, GeometricMean: 92.00%, CohenKappa: 85.40%
[2,000] Accuracy: 94.60%, BalancedAccuracy: 93.48%, GeometricMean: 93.42%, CohenKappa: 87.85%
[2,500] Accuracy: 93.60%, BalancedAccuracy: 93.00%, GeometricMean: 92.98%, CohenKappa: 85.44%
[3,000] Accuracy: 93.80%, BalancedAccuracy: 92.81%, GeometricMean: 92.76%, CohenKappa: 86.00%
[3,500] Accuracy: 93.60%, BalancedAccuracy: 93.03%, GeometricMean: 93.02%, CohenKappa: 85.15%
[4,000] Accuracy: 96.20%, BalancedAccuracy: 96.03%, GeometricMean: 96.03%, CohenKappa: 91.28%
[4,500] Accuracy: 93.60%, BalancedAccuracy: 92.56%, GeometricMean: 92.50%, CohenKappa: 85.62%
[5,000] Accuracy: 93.80%, BalancedAccuracy: 92.90%, GeometricMean: 92.86%, CohenKappa: 86.17%
[5,500] Accuracy: 57.60%, BalancedAccuracy: 56.61%, GeometricM

Accuracy: 65.00%, BalancedAccuracy: 55.39%, GeometricMean: 37.54%, CohenKappa: 12.69%